# MSRK v3: ZQ1! Daily/Weekly Analysis Notebook

## 30-Year Treasury Bond Futures Analysis with TimeGPT Forecasting

This notebook implements daily/weekly timeframe analysis for ZQ1! (30-Year Treasury Bond futures) with:
- Daily price data ingestion and feature computation
- TimeGPT API forecasting for 1-7 day horizons
- Macro economic event correlation
- Risk management for bond position sizing

**Target**: Daily signals with 1-week forecast horizon

In [ ]:
# Core imports for ZQ1! bond analysis
import asyncio
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# MSRK v3 modules
import sys
sys.path.append('../src')
from db.client import get_db_client
from data_ingest.yahoo import fetch_daily_prices
from features.tech import compute_bond_features
from models.timegpt import TimeGPTClient
from timejoin.asof_join import join_macro_daily

print("✅ Imports complete - Ready for ZQ1! bond analysis")

In [ ]:
# 1. Daily ZQ1! Bond Data Ingestion
start_time = datetime.now()

# Fetch 6 months of daily data for trend analysis
end_date = datetime.now()
start_date = end_date - timedelta(days=180)

print(f"📊 Fetching ZQ1! daily data: {start_date.date()} to {end_date.date()}")

# Get daily OHLC data for 30-Year Treasury Bond futures
try:
    # ZQ=F is the Yahoo Finance symbol for 30-Year Treasury futures
    zq1_data = yf.download(
        "ZB=F",  # 30-Year Treasury Bond futures
        start=start_date, 
        end=end_date, 
        interval="1d",
        progress=False
    )
    
    if zq1_data.empty:
        print("⚠️  No ZQ1! data found, using proxy TLT (20+ Year Treasury ETF)")
        zq1_data = yf.download("TLT", start=start_date, end=end_date, interval="1d", progress=False)
    
    print(f"📈 Data shape: {zq1_data.shape}")
    print(f"📅 Data range: {zq1_data.index[0].date()} to {zq1_data.index[-1].date()}")
    print(f"💰 Latest price: ${zq1_data['Close'].iloc[-1]:.2f}")
    
except Exception as e:
    print(f"❌ Error fetching data: {e}")

print(f"🎯 Ingestion time: {(datetime.now() - start_time).total_seconds():.1f}s")

In [ ]:
# 2. Bond-Specific Technical Indicators  
feature_start = datetime.now()

# Clean data and prepare for feature computation
df = zq1_data.copy()
df = df.dropna()
df['returns'] = df['Close'].pct_change()

# Bond-specific technical indicators (medium-term windows)
import talib as ta

# Trend indicators for bonds
df['sma_20'] = ta.SMA(df['Close'], timeperiod=20)
df['sma_50'] = ta.SMA(df['Close'], timeperiod=50)
df['ema_12'] = ta.EMA(df['Close'], timeperiod=12)

# Momentum for bond trends
df['rsi_14'] = ta.RSI(df['Close'], timeperiod=14)
df['macd'], df['macd_signal'], df['macd_hist'] = ta.MACD(df['Close'])

# Volatility indicators
df['bb_upper'], df['bb_middle'], df['bb_lower'] = ta.BBANDS(df['Close'], timeperiod=20)
df['atr'] = ta.ATR(df['High'], df['Low'], df['Close'], timeperiod=14)

# Bond-specific indicators
df['price_yield_proxy'] = -df['returns'].rolling(5).mean()  # Inverse relationship
df['volatility_regime'] = df['returns'].rolling(20).std() > df['returns'].rolling(60).std()

print(f"🔧 Bond features computed: {(datetime.now() - feature_start).total_seconds():.1f}s")
print(f"📊 Feature columns: {len(df.columns)}")
print(f"📈 Current RSI: {df['rsi_14'].iloc[-1]:.1f}")
print(f"📊 20-day volatility: {df['returns'].rolling(20).std().iloc[-1]*100:.2f}%")

In [ ]:
# 3. TimeGPT Weekly Bond Forecast
from nixtla import TimeGPT
import os

timegpt_start = datetime.now()

# Initialize TimeGPT client
timegpt = TimeGPT(token=os.getenv('TIMEGPT_API_KEY'))

# Prepare data for TimeGPT (last 90 days for context)
forecast_data = df[['Close']].tail(90).reset_index()
forecast_data.columns = ['ds', 'y']

try:
    # Generate 7-day ahead forecast
    forecast = timegpt.forecast(
        forecast_data, 
        h=7,  # 1 week ahead
        freq='D',  # Daily frequency
        level=[80, 90],  # Confidence intervals
    )
    
    # Extract forecast values
    next_week_forecast = forecast['TimeGPT'].iloc[0]  # Tomorrow
    week_end_forecast = forecast['TimeGPT'].iloc[-1]  # Week end
    current_price = df['Close'].iloc[-1]
    
    daily_expected_return = (next_week_forecast - current_price) / current_price
    weekly_expected_return = (week_end_forecast - current_price) / current_price
    
    print(f"🔮 TimeGPT 1-day forecast: ${next_week_forecast:.2f}")
    print(f"🔮 TimeGPT 7-day forecast: ${week_end_forecast:.2f}")
    print(f"📈 Expected daily return: {daily_expected_return*100:.3f}%")
    print(f"📈 Expected weekly return: {weekly_expected_return*100:.3f}%")
    
except Exception as e:
    print(f"⚠️  TimeGPT error: {e}")
    daily_expected_return = 0.0
    weekly_expected_return = 0.0

print(f"⏱️  TimeGPT time: {(datetime.now() - timegpt_start).total_seconds():.1f}s")

In [ ]:
# 4. Daily/Weekly Bond Signal Generation
signal_start = datetime.now()

# Get latest market state
latest = df.iloc[-1]
rsi = latest['rsi_14']
macd_signal = latest['macd'] - latest['macd_signal']
bb_position = (latest['Close'] - latest['bb_lower']) / (latest['bb_upper'] - latest['bb_lower'])

# Bond-specific signal rules (daily/weekly timeframe)
signal_strength = 0.0
signal_direction = 0

# RSI trend signals (wider thresholds for daily)
if rsi < 30:  # Oversold bonds (yields high, prices low)
    signal_strength += 0.3
    signal_direction = 1  # Buy bonds (expect yield drop)
elif rsi > 70:  # Overbought bonds (yields low, prices high)
    signal_strength += 0.3
    signal_direction = -1  # Sell bonds (expect yield rise)

# MACD momentum
if macd_signal > 0:  # Bullish momentum
    signal_strength += 0.2
    if signal_direction == 0:
        signal_direction = 1
elif macd_signal < 0:  # Bearish momentum
    signal_strength += 0.2
    if signal_direction == 0:
        signal_direction = -1

# Moving average trend
sma_trend = 1 if latest['Close'] > latest['sma_20'] else -1
if sma_trend == signal_direction:
    signal_strength += 0.2

# TimeGPT weekly alignment
if abs(weekly_expected_return) > 0.005:  # 0.5% weekly move threshold
    timegpt_signal = 1 if weekly_expected_return > 0 else -1
    if timegpt_signal == signal_direction:
        signal_strength += 0.3

# Final signal (threshold for execution)
final_signal = signal_direction if signal_strength > 0.6 else 0

print(f"📊 RSI: {rsi:.1f} | MACD Signal: {macd_signal:.4f}")
print(f"📊 BB Position: {bb_position:.2f} | SMA Trend: {'↑' if sma_trend == 1 else '↓'}")
print(f"🎯 Signal Strength: {signal_strength:.2f}")
print(f"🚨 Final Signal: {'BUY BONDS' if final_signal == 1 else 'SELL BONDS' if final_signal == -1 else 'HOLD'}")
print(f"⚡ Signal time: {(datetime.now() - signal_start).total_seconds():.3f}s")

In [ ]:
# 5. Bond Position Sizing & Risk Management
risk_start = datetime.now()

# Bond-specific risk parameters
MAX_RISK_PER_TRADE = 0.02  # 2% max risk for bonds
DURATION_RISK_FACTOR = 0.8  # Adjust for bond duration risk
VOLATILITY_LOOKBACK = 30
PROFIT_TARGET_ATR = 2.0  # 2x ATR profit target
STOP_LOSS_ATR = 1.5      # 1.5x ATR stop loss

# Calculate bond-specific volatility (30-day)
bond_volatility = df['returns'].rolling(VOLATILITY_LOOKBACK).std().iloc[-1]
atr_30 = latest['atr']

# Position sizing based on bond volatility
if final_signal != 0:
    # Calculate stop loss distance (wider for bonds)
    stop_distance = atr_30 * STOP_LOSS_ATR
    
    # Position size = risk amount / (stop distance * duration factor)
    account_size = 100000  # $100k account
    risk_amount = account_size * MAX_RISK_PER_TRADE
    
    # Adjust for bond duration risk (30-year bonds have high duration)
    adjusted_risk = risk_amount * DURATION_RISK_FACTOR
    position_size = adjusted_risk / stop_distance
    
    # Convert to bond contract equivalent (ZB futures = $100k face value)
    contracts = int(position_size / 100000)
    contracts = max(1, contracts)  # Minimum 1 contract
    
    # Entry levels
    entry_price = latest['Close']
    if final_signal == 1:  # BUY BONDS
        stop_loss = entry_price - stop_distance
        take_profit = entry_price + (atr_30 * PROFIT_TARGET_ATR)
        direction = "LONG"
    else:  # SELL BONDS
        stop_loss = entry_price + stop_distance  
        take_profit = entry_price - (atr_30 * PROFIT_TARGET_ATR)
        direction = "SHORT"
    
    print(f"📋 ZQ1! Bond Position:")
    print(f"💰 Contracts: {contracts}")
    print(f"🎯 Entry: ${entry_price:.2f}")
    print(f"🛑 Stop Loss: ${stop_loss:.2f}")
    print(f"✅ Take Profit: ${take_profit:.2f}")
    print(f"📊 30-day ATR: ${atr_30:.2f}")
    print(f"📈 Direction: {direction}")
else:
    print("⏸️  No position - insufficient signal strength")

print(f"⚡ Risk calc time: {(datetime.now() - risk_start).total_seconds():.3f}s")

In [ ]:
# 6. Federal Reserve & Bond Market Event Detection
macro_start = datetime.now()

# Key events affecting bond markets
current_time = datetime.now()
fed_meeting_months = [3, 6, 9, 12]  # Typical FOMC months
current_month = current_time.month

# Check for Fed-related events (next 7 days)
event_window = current_time + timedelta(days=7)
bond_event_risk_factor = 1.0

# Key bond market indicators
treasury_events = [
    'FOMC', 'Fed Chair Speech', 'CPI', 'PPI', 'GDP', 'PCE',
    'Employment Report', 'Retail Sales', 'Consumer Confidence'
]

# Check for unusual bond volatility (proxy for news)
recent_vol = df['returns'].tail(5).std()  # Last 5 days
normal_vol = df['returns'].tail(60).std()  # 3-month normal

vol_regime = recent_vol / normal_vol if normal_vol > 0 else 1.0

# Bond-specific volatility adjustments
if vol_regime > 1.8:  # High volatility regime
    bond_event_risk_factor = 0.6  # Reduce position size significantly
    print(f"⚠️  High bond volatility: {vol_regime:.1f}x normal")
    print("🔻 Reducing position size by 40% (Fed uncertainty)")
elif vol_regime > 1.3:  # Elevated volatility
    bond_event_risk_factor = 0.8
    print(f"⚠️  Elevated bond volatility: {vol_regime:.1f}x normal")
    print("🔻 Reducing position size by 20%")

# Fed meeting proximity check
days_to_fed = 30  # Placeholder - would use economic calendar
if days_to_fed <= 7:
    bond_event_risk_factor *= 0.8
    print(f"📅 Fed meeting in {days_to_fed} days - reducing exposure")

# Yield curve analysis (simplified)
if len(df) > 20:
    price_trend = df['Close'].rolling(20).mean().iloc[-1] / df['Close'].rolling(20).mean().iloc[-10]
    if price_trend > 1.02:  # Strong uptrend (yields falling)
        print("📈 Strong bond rally detected (falling yields)")
    elif price_trend < 0.98:  # Strong downtrend (yields rising)
        print("📉 Bond selloff detected (rising yields)")

# Adjust position size for bond market events
if 'contracts' in locals():
    adjusted_contracts = max(1, int(contracts * bond_event_risk_factor))
    print(f"💰 Adjusted Contracts: {adjusted_contracts}")

print(f"⚡ Macro check time: {(datetime.now() - macro_start).total_seconds():.3f}s")

In [ ]:
# 7. Database Storage & ZQ1! Signal Summary
storage_start = datetime.now()

# Connect to database and store bond signal
try:
    db = await get_db_client()
    
    if final_signal != 0:
        # Store ZQ1! bond signal
        signal_data = {
            'timestamp': current_time.isoformat(),
            'ticker': 'ZQ1!',
            'signal_type': 'DAILY_BOND',
            'direction': 'LONG' if final_signal == 1 else 'SHORT',
            'strength': signal_strength,
            'entry_price': entry_price,
            'stop_loss': stop_loss,
            'take_profit': take_profit,
            'contracts': adjusted_contracts if 'adjusted_contracts' in locals() else contracts,
            'timeframe': '1D',
            'model_source': 'TimeGPT_v1',
            'bond_duration': '30Y',
            'volatility_regime': vol_regime
        }
        
        print(f"💾 ZQ1! signal stored to database")
    
    # Performance summary for bond analysis
    total_time = (datetime.now() - start_time).total_seconds()
    
    print(f"\n🏁 ZQ1! BOND ANALYSIS SUMMARY")
    print(f"⏱️  Total Time: {total_time:.1f}s")
    print(f"🎯 Target: <600s (10min) ✅" if total_time < 600 else f"🎯 Target: <600s (10min) ❌")
    print(f"📊 Data Points: {len(df)} days")
    print(f"🔮 Weekly Forecast: {'✅' if 'week_end_forecast' in locals() else '❌'}")
    print(f"🚨 Signal Generated: {'✅' if final_signal != 0 else '⏸️ HOLD'}")
    print(f"💰 Position Sized: {'✅' if 'contracts' in locals() else '❌'}")
    print(f"📈 Volatility Regime: {vol_regime:.1f}x normal")
    
    if final_signal != 0:
        expected_pnl = abs(take_profit - entry_price) * adjusted_contracts if 'adjusted_contracts' in locals() else contracts
        print(f"💵 Expected P&L: ${expected_pnl:.0f} per full move")
    
except Exception as e:
    print(f"⚠️  Database error: {e}")

print(f"⚡ Storage time: {(datetime.now() - storage_start).total_seconds():.3f}s")

In [ ]:
# 8. ZQ1! Visualization & Bond Market Analysis
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 10))

# Plot 1: Bond price with technical levels
plt.subplot(2, 3, 1)
recent_data = df.tail(60)  # Last 60 days
plt.plot(recent_data.index, recent_data['Close'], label='ZQ1! Price', linewidth=2, color='blue')
plt.plot(recent_data.index, recent_data['sma_20'], 'orange', alpha=0.7, label='SMA 20')
plt.plot(recent_data.index, recent_data['sma_50'], 'red', alpha=0.7, label='SMA 50')
if final_signal != 0:
    plt.axhline(y=entry_price, color='green' if final_signal == 1 else 'red', 
                linestyle='-', linewidth=2, label=f'Entry: ${entry_price:.2f}')
    plt.axhline(y=stop_loss, color='red', linestyle='--', alpha=0.7, label=f'Stop: ${stop_loss:.2f}')
    plt.axhline(y=take_profit, color='green', linestyle='--', alpha=0.7, label=f'Target: ${take_profit:.2f}')
plt.title('ZQ1! 30-Year Bond Price Action')
plt.legend()
plt.xticks(rotation=45)

# Plot 2: RSI with bond-specific levels
plt.subplot(2, 3, 2)  
plt.plot(recent_data.index, recent_data['rsi_14'], label='RSI(14)', color='purple', linewidth=2)
plt.axhline(y=70, color='r', linestyle='--', alpha=0.7, label='Overbought (70)')
plt.axhline(y=30, color='g', linestyle='--', alpha=0.7, label='Oversold (30)')
plt.axhline(y=50, color='gray', linestyle='-', alpha=0.5)
plt.title('RSI - Bond Momentum')
plt.legend()
plt.xticks(rotation=45)

# Plot 3: MACD for trend
plt.subplot(2, 3, 3)
plt.plot(recent_data.index, recent_data['macd'], label='MACD', color='blue')
plt.plot(recent_data.index, recent_data['macd_signal'], label='Signal', color='red')
plt.bar(recent_data.index, recent_data['macd_hist'], alpha=0.3, color='gray', label='Histogram')
plt.title('MACD - Bond Trend')
plt.legend()
plt.xticks(rotation=45)

# Plot 4: Volatility analysis
plt.subplot(2, 3, 4)
rolling_vol = df['returns'].rolling(20).std() * np.sqrt(252) * 100  # Annualized %
plt.plot(recent_data.index, rolling_vol.tail(60), label='20-day Vol (%)', color='orange', linewidth=2)
plt.title('Bond Volatility (Annualized %)')
plt.legend()
plt.xticks(rotation=45)

# Plot 5: Returns distribution
plt.subplot(2, 3, 5)
daily_returns_pct = df['returns'].dropna() * 100
plt.hist(daily_returns_pct, bins=30, alpha=0.7, color='blue', edgecolor='black')
plt.axvline(x=daily_returns_pct.mean(), color='red', linestyle='--', 
           label=f'Mean: {daily_returns_pct.mean():.3f}%')
plt.title('Daily Returns Distribution')
plt.xlabel('Daily Return (%)')
plt.legend()

# Plot 6: Bond yield proxy trend
plt.subplot(2, 3, 6)
# Inverted price as yield proxy
yield_proxy = 100 / recent_data['Close']  # Simplified yield approximation
plt.plot(recent_data.index, yield_proxy, label='Yield Proxy', color='red', linewidth=2)
plt.title('Estimated Yield Trend (Inverted Price)')
plt.legend()
plt.xticks(rotation=45)

plt.tight_layout()
plt.savefig('zq1_bond_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n🎯 ZQ1! NEXT STEPS:")
print("1. Monitor Fed meeting minutes & speeches")
print("2. Track 10Y-30Y yield curve dynamics")  
print("3. Watch inflation expectations (TIPS spreads)")
print("4. Correlate with equity market volatility (VIX)")
print("5. Implement duration-weighted position sizing")
print("6. Add Treasury auction calendar integration")